<h1><center>Laboratorio 7: La desperación de Mr. Lepin 🐼</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angel Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Ricardo Torres
- Nombre de alumno 2: Simon Estrada


### **Link de repositorio de GitHub:** `http://....`

### Indice

1. [Temas a tratar](#Temas-a-tratar:)
3. [Descripción del laboratorio](#Descripción-del-laboratorio.)
4. [Desarrollo](#Desarrollo)

# Temas a tratar

- Aplicar Pandas para obtener características de un DataFrame.
- Aplicar Pipelines.
- Aplicar Clusters sobre un conjunto de datos.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Comprender y aprovechar las ventajas que nos ofrece la librería `pandas` con respecto a trabajar en Python 'puro'.
- Crear nuevas características para entrenar un modelo de clustering.
- Comprender como aplicar pipelines de Scikit-Learn para generar procesos más limpios.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `numpy`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre arreglos (*o tensores*).

## Descripción del laboratorio.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
import datetime
from IPython.display import HTML

# Libreria para plotear (En colab esta desactualizado plotly)
!pip install --upgrade plotly
import plotly.express as px
import plotly.graph_objects as go

# Librerias utiles
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 73.2 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except:
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


# Segmentación de Clientes en Tienda de Retail 🛍️

<p align="center">
  <img width=300 src="https://s1.eestatic.com/2018/04/14/social/la_jungla_-_social_299733421_73842361_854x640.jpg">
</p>

## 1.1 Cargar Dataset

Mr. Lepin, en una nueva reunión, le cuenta a ud y su equipo que los resultados derivados del análisis exploratorio de dato presentaron una gran utilidad para la empresa y que tiene un gran entusiasmo por continuar trabajando con ustedes.
Es por esto, que Mr. Lepin les pide que cargue y visualicen algunas de las filas que componen el Dataset.
A continuación un extracto de lo parlamentado en la reunión:

    - Usted: Es un gran logro para nuestro equipo que usted haya encontrado excelente el EDA. ¿Qué tiene en mente ahora?
    - Mr. Lepin: Resulta que hace algún tiempo, mientras tomaba un mojito en una reunión de gerentes en Panamá, oí a un *chato* acerca de **LRMFP**, que es un modelo que permite personificar a los clientes a través de la fabricación de distintos atributos que describen a los clientes. Lo encontré es-tu-pendo ñatito.
    - Usted: Ehh bueno. Investigaremos acerca de este modelo y veremos lo que podemos hacer.

Por ende, su siguiente tarea es calcular **LRMFP** sobre cada cliente y luego hacer un análisis de las características generadas. Para esto, el área de ventas les entrega un nuevo archivo llamado `online_retail_II_cleaned.pickle`, quien posee los datos del DataFrame original limpios y listos para obtener las características solicitadas por Mr. Lepin.

In [ ]:
cd /content/drive/MyDrive/Lab_07/

/content/drive/MyDrive/Lab_07


In [ ]:
df_retail = pd.read_pickle("online_retail_II_cleaned.pickle")
df_retail = df_retail.astype(
    {
        "Invoice": "category",
        "StockCode": "category",
        "Description": "category",
        "Description": str,
        "Customer ID": "category",
        "Country": "category"
    }
)
df_retail.head()


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


## 1.2 Creación de nuevas Caracteristicas [2 Puntos]

Como ya se les comento, Mr. Lepin esta interesado en obtener las características **LRMFP**, para esto les señala que estas características se construyen en base a las siguientes definiciones:

- **Length (L)**: Intervalo de tiempo, en días, entre la primera y la última visita del cliente. Mientras mas grande sea el valor, mas fiel es el cliente.

- **Recency (R)**: Indica hace cuanto tiempo el cliente realizo su ultima compra. Notar que para este caso, mientras mas grande es el valor, menos interes posee el usuario para repetir una compra en uno de los locales.

- **Monetary (M)**: El término "monetario" se refiere a la cantidad media de dinero gastada por cada visita del cliente durante el período de observación y refleja la contribución del cliente a los ingresos de la empresa.

- **Frequency (F)**: Se refiere al número total de visitas del cliente durante el periodo de observación. Cuanto mayor sea la frecuencia, mayor será la fidelidad del cliente.

- **Periodicity (P)**: Representa si los clientes visitan las tiendas con regularidad.

$$Periodicity(n)=std(IVT_1, ..., IVT_n)$$

&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Donde $IVT$ denota el tiempo entre visitas y n representa el número de valores de tiempo entre visitas de un cliente.


$$IVT_i=date\_diff(t_{i+1},t)$$

En base a las definiciones señaladas, diseñe una función que permita obtener las características **LRMFP** recibiendo un DataFrame como entrada. Para esto, no estará permitido el uso de iteradores, utilice todas las herramientas que les ofrece `pandas` para realizar esto.

Una referencia que le puede ser útil es el [documento original](https://www.researchgate.net/publication/315979555_LRFMP_model_for_customer_segmentation_in_the_grocery_retail_industry_a_case_study) en donde se propone este método.

In [ ]:
def custom_features(dataframe_in):

  result= dataframe_in.copy()

  # Encuentra la última fecha de conexión (InvoiceDate) por Customer ID
  last_purchase = dataframe_in.groupby('Customer ID')['InvoiceDate'].max()

  # Encuentra la primera fecha de conexión (InvoiceDate) por Customer ID
  first_purchase = dataframe_in.groupby('Customer ID')['InvoiceDate'].min()

  Length=  (last_purchase - first_purchase).dt.days

  max_date = dataframe_in['InvoiceDate'].max()
  # Suma un día a la fecha máxima
  actual_day = max_date + pd.Timedelta(days=1)

  Recency = (actual_day - dataframe_in.groupby('Customer ID')['InvoiceDate'].max()).dt.days

  # Calcular la frecuencia (Frequency)
  Frequency = dataframe_in.groupby('Customer ID')['InvoiceDate'].nunique()

  # Calcular el valor monetario (Monetary) considerando la multiplicación de Price y Quantity
  dataframe_in['Monetary'] = dataframe_in['Price'] * dataframe_in['Quantity']

  # Calcular el Monetary (M)
  Monetary =dataframe_in.groupby('Customer ID')['Monetary'].mean()

  # Calcular los IVT (intervalos de tiempo entre visitas)
  dataframe_in['IVT'] = dataframe_in.groupby('Customer ID')['InvoiceDate'].diff()
  Periodicity = dataframe_in['IVT'].dt.days.groupby(dataframe_in['Customer ID']).std()

  # Crear un DataFrame con las columnas 'Customer ID', 'Lenght', y 'Recency'
  result = pd.DataFrame({'Length': Length, 'Recency': Recency,'Frequency':Frequency, 'Monetary': Monetary, 'Periodicity': Periodicity}).reset_index()

  return result

**<u>Formato</u> del Resultado Esperado:**

| Customer ID | Length | Recency | Frequency | Monetary | Periodicity |
|------------:|-------:|--------:|----------:|---------:|------------:|
|   12346.0   |    294 |      67 |        46 |   -64.68 |        37.0 |
|   12347.0   |     37 |       3 |        71 |  1323.32 |         0.0 |
|   12349.0   |    327 |      43 |       107 |  2646.99 |        78.0 |
|   12352.0   |     16 |      11 |        18 |   343.80 |         0.0 |
|   12356.0   |     44 |      16 |        84 |  3562.25 |        12.0 |

**Resúesta de Pauta:**

In [ ]:
custom_features(df_retail).head(10)

,Customer ID,Length,Recency,Frequency,Monetary,Periodicity
0,12346.0,196,165,11,11.298788,21.724076
1,12347.0,37,3,2,18.638310,4.422346
2,12348.0,0,74,1,11.108000,0.000000
3,12349.0,181,43,3,26.187647,16.200990
4,12351.0,0,11,1,14.330000,0.000000
5,12352.0,16,11,2,19.100000,3.880570
6,12353.0,0,44,1,15.888000,0.000000
7,12355.0,0,203,1,22.191364,0.000000
8,12356.0,44,16,3,42.895181,3.655625
9,12357.0,0,24,2,73.212061,0.000000


## 1.3 Pipelines 👷

Finalmente *Don Mora* le pregunta si seria posible realizar un pipeline para realizar una segmentación de los clientes con los nuevos datos generados, a lo que usted responde que **sí** y propone la utilización de k-means para la segmentación.

A continuación siga los pasos requeridos para obtener la segmentación de clientes.

### 1.3.1 Estandarizar Caracteristicas [0.5 puntos]

Construya una clase llamada ``MinMax()`` utilizando ``BaseEstimator`` y ``TransformerMixin`` para realizar una transformación de cada una de las columnas de un DataFrame utilizando ``ColumnTransformer()`` más tarde (tome como referencia el siguiente [enlace](https://sklearn-template.readthedocs.io/en/latest/user_guide.html#transformer)).


 Para esto considere que Min-Max escaler queda dada por la ecuación:

$$MinMax = \dfrac{x-min(x)}{max(x) - min(x)}$$

Con esto buscamos que los valores que componen a las columnas se muevan en el rango de valores $[0, 1]$.

**Respuesta:**

In [ ]:
class MinMax(BaseEstimator, TransformerMixin):
    def fit(self, X):
        self.min_vals = X.min()
        self.max_vals = X.max()
        return self

    def transform(self, X):
        # Aplicar la transformación Min-Max a cada columna
        X_transformed = (X - self.min_vals) / (self.max_vals - self.min_vals)

        return X_transformed


### 1.3.2 `T-SNE` Pipeline [1.0 puntos]

Para comenzar introduciéndose en el uso de pipeline, decide probar realizando un pipeline enfocado en la reducción de dimensionalidad y así hacer no decepcionar a Mr. Lepin con la clusterización del modelo.

Configure un pipeline utilizando el algoritmo `T-SNE` sobre los datos **LRMFP**, donde, para la realización del pipeline considera los siguientes pasos:

1. Como primer paso obtenga las características **LRMFP** desde el DataFrame ``df_retail_II_cleaned.pickle`` utilizando la función ``custom_features`` creada anteriormente, junto a ``FunctionTransformer()``. Considere esto como el primer paso de su pipeline.
2. En segundo lugar usando ``ColumnTransformer()`` aplique el MinxMax scaler creado por usted sobre todas las columnas generadas en el paso anterior.
3. Finalmente, aplique un último paso donde obtiene las 2 componentes más relevantes utilizando el algoritmo `T-SNE` de sckit-learn.

Tras aplicar las transformaciones sobre el dataset **LRMFP**, gráfique las componentes obtenidas en la reducción de dimensionalidad.

**Respuesta:**

In [ ]:
custom_features(df_retail).isna().sum()

Customer ID      0
Length           0
Recency          0
Frequency        0
Monetary         0
Periodicity    148
dtype: int64

In [ ]:
#importamos el imputador
from sklearn.impute import SimpleImputer

# Paso 1: Usando FunctionTransformer para convertir la función en un transformador
feature_extractor = FunctionTransformer(custom_features, validate=False)

#Imputador: Necesario debido a que custom_features existen valores NaN en "Periodicity"
imputer = SimpleImputer(strategy='mean')

transformer = ColumnTransformer(
    transformers=[('"MinMax"', MinMax(),slice(0,None))],  #slice para seleccionar todas las columnas excepeto la primera
    remainder='passthrough'
)

# #Paso 3:T-SNE
tsne = TSNE(n_components=2)

#Se crea el pipline
pipeline = Pipeline([
    ('features', feature_extractor),
    ('impute', imputer),
    ('scaler', transformer),
    ('dimension_reduction', tsne)
])

In [ ]:
# Ajustar el pipeline a los datos
reduced_features = pipeline.fit_transform(df_retail)

# Crear un DataFrame con las componentes reducidas
reduced_df = pd.DataFrame(data=reduced_features, columns=["Componente 1", "Componente 2"])

fig = px.scatter(
    reduced_df, x="Componente 1", y="Componente 2",
    title="Gráfico de Dispersión de Componentes Reducidas (T-SNE)",
    labels={'Componente 1': 'TSNE Componente 1', 'Componente 2': 'TSNE Componente 2'},
    width=800, height=600
)

fig.show()

### 1.3.3 Clustering

#### 1.3.3.1 Método del Codo [1 puntos]

Utilizando la clase creada para escalamiento, aplique el método del codo para visualizar cual es el número de clusters que mejor se ajustan a los datos. Realice esto utilizando el algoritmo K-means dentro de un pipeline para un $k \in [1,20]$, donde k representa el número de clusters del k-means. Para la realización de esta sección y la próxima (1.3.3.2), considere los mismos pasos utilizados para el t-sne, pero **permutando el algoritmo de reducción de dimensionalidad por k-means.**

A través del grafico obtenido, comente y justifique que valor de k escogería para realizar el k-means.

**Respuesta:**

In [ ]:
# Números de clusters a probar
k_values = range(1, 21)
inertia_values = []

for k in k_values:
    # Crear un nuevo pipeline con K-Means y el número de clusters actual
    kmeans = KMeans(n_clusters=k, random_state=42)
    cluster_pipeline = Pipeline([
        ('features', feature_extractor),
        ('impute', imputer),
        ('scaler', transformer),
        ('clustering', kmeans)
    ])

    # Aplicar el pipeline y obtener la Inertia
    cluster_pipeline.fit_transform(df_retail)
    inertia = cluster_pipeline.named_steps['clustering'].inertia_
    inertia_values.append(inertia)

# Crear un DataFrame con los valores de k y la Inertia
elbow_data = pd.DataFrame({'k': k_values, 'Inertia': inertia_values})

# Crear un gráfico interactivo con Plotly
fig = px.line(elbow_data, x='k', y='Inertia', title='Método del Codo para Determinar el Valor Óptimo de k')
fig.update_traces(mode='lines+markers')
fig.update_xaxes(title='Número de Clusters (k)')
fig.update_yaxes(title='Inertia')
fig.show()

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will chang

Con el grafico obtenido y siguiendo la regla del codo un buen numero de clusters para kmeans correpsonderia a 5.

#### 1.3.3.2 Segmentación de Clientes con K-Means 🎁 [1 punto]

En base a la elección de k realizada en la sección anterior, utilice este valor escogido y entrene un modelo de K-means utilizando el mismo pipeline de scikit-learn utilizado anteriormente.

Una vez ajustado los datos, genere una tabla con los promedios (o medianas) para cada uno de los atributos, agrupando estos por el clúster que pertenecen. ¿Es posible observar agrupaciones coherentes?, ¿Qué tipo de clientes posee el retail?, Justifique su respuesta y no decepcione a Mr. Lepin.


**Respuesta:**

In [ ]:
# Número óptimo de clusters (elegido en la sección anterior)
optimal_k = 5  # Reemplaza con el valor óptimo de k que obtuviste

# Crear un nuevo K-Means con el número óptimo de clusters
kmeans = KMeans(n_clusters=optimal_k, random_state=42)

# Crear un nuevo pipeline con K-Means
cluster_pipeline = Pipeline([
    ('features', feature_extractor),
    ('impute', imputer),
    ('scaler', transformer),
    ('clustering', kmeans)
])

# Ajustar el modelo a los datos
cluster_pipeline.fit_transform(df_retail)

# Obtener las etiquetas de clústeres
labels = cluster_pipeline.named_steps['clustering'].labels_

# Crear una copia del DataFrame original para evitar errores de longitud
df_retail_copy = custom_features(df_retail).drop("Customer ID", axis=1)

# Aplicar el imputador al DataFrame copiado
imputed_df = imputer.fit_transform(df_retail_copy)

# Convertir el resultado en un DataFrame
imputed_df = pd.DataFrame(imputed_df, columns=df_retail_copy.columns)

# Agregar las etiquetas de clústeres al DataFrame copiado
imputed_df['Cluster'] = labels

# Calcular los promedios para cada atributo agrupados por clúster
cluster_means = imputed_df.groupby('Cluster').agg({
    'Length': 'mean',
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean',
    'Periodicity': 'mean',
})

# Mostrar la tabla de promedios
cluster_means

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



,Length,Recency,Frequency,Monetary,Periodicity
Cluster,,,,,
0,130.701587,92.666667,4.526984,33.557084,11.797133
1,144.391667,86.950000,4.434524,34.679788,12.730180
2,131.451422,97.939573,4.580569,38.711893,12.254135
3,135.013468,86.783389,4.166105,28.020891,11.135362
4,128.158690,92.118388,4.531486,51.737722,11.054306


**Respuesta Esperada:**

|         | Length  | Recency   | Frequency | Monetary | Periodicity |       |
|---------|---------|-----------|----------|-------------|-------|-------|
| Cluster |         |           |          |             |       |       |
|    0    |   258.8 |      45.2 |     76.1 |      1107.7 | 107.6 |   449 |
|    1    |    76.1 |     217.6 |     45.5 |       791.7 |  14.1 |   466 |
|    2    |   368.5 |       4.8 |   2715.0 |    226621.6 |   4.2 |     4 |
|    3    |    85.3 |      45.7 |     65.8 |      1047.0 |  10.5 |   987 |
|    4    |   347.2 |      15.9 |   1658.0 |     35829.3 |   8.0 |    25 |
|    5    |   298.0 |      29.8 |    183.8 |      3639.9 |  32.0 |  1188 |

Analizando la tabla obtenida, es posible reconocer clusters bastante representativos y coherentes en los datos. En general, debido al número de clusters, se podrían dividir preliminarmente en 5 tipos de clientes: La variable con mayor variación entre estos tipos de clientes correspondería a la diferencia en largos entre la primera y última visita del cliente, es decir, qué tan fieles son estos clientes.

Después de separar por fidelidad, teniendo un cluster de clientes más fieles en el cluster 1, se observa que la siguiente diferencia se encuentra vinculada a la recencia, lo cual indica cuánto tiempo ha pasado desde la última visita del cliente. De esto se pueden separar dos grupos de clientes: los que visitaron hace 86 días y los que visitaron hace más de 90 días, siendo el cluster 1 y el 3 los que visitaron más recientemente en promedio.

En términos de frecuencia, no hay mucha variación entre los clusters, excepto en el cluster 3, que corresponde a un grupo que no visita la empresa tan frecuentemente. En cuanto a lo monetario, se observa que gran parte de los clusters se encuentran cercanos en términos de cuánto dinero gastan en la empresa, excepto el cluster 4, cuyos clientes gastan considerablemente más que el promedio de los otros clusters. Claro que también corresponden a un cluster de clientes mucho más recientes, y además, la última vez que visitaron la empresa fue hace más tiempo que en otros clusters.

Finalmente, en cuanto a la periodicidad, se podría decir que los clientes que gastan más son los que visitan la tienda menos regularmente. Además, el cluster 1, que tenía los clientes más fieles, visita la empresa bastante regularmente.

#### 1.3.3.3 Plot de K-Means 📈 [0.5 puntos]



Por último, Mr. Lepin, impaciente de no entender lo que usted intenta explicarle, le solicita que por favor muestre algún resultado "visual" de los grupos encontrados.

Para esto, grafique nuevamente las características encontradas usando `T-SNE` (no calcule de nuevo, simplemente utilice las proyecciones encontradas) y agregue las labels calculadas con kmeans como el argumento `color`.   

Comente: ¿Se separan bien los distintos clusters en la visualización?


**Respuesta:**

In [ ]:
fig = px.scatter(
    reduced_df, x="Componente 1", y="Componente 2", color=labels,
    title="Gráfico de Dispersión de Componentes Reducidas (T-SNE)",
    labels={'Componente 1': 'TSNE Componente 1', 'Componente 2': 'TSNE Componente 2'},
    width=800, height=600
)

fig.show()

En la visualizacion obtenida, se observa que los clusters se separan relativamente bien en ciertas partes como en la izquieda y la derecha. Pero se evidencia que los clusters obtenidos no separan de forma perfecta las distintas etiquetas de la data en partes como el cluster central, donde hay un gran numero de puntos que empiezan a mesclarse un poco. En general la separaciones es bastante buena a excepción de las junturas entre clusters.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por correo, Discord o U-cursos.

![Gracias Totales!](https://i.pinimg.com/originals/65/ae/27/65ae270df87c3c4adcea997e48f60852.gif "bruno")


<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>